In [1]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Geometry import rdGeometry
import nglview as nv
from ipywidgets import interactive, VBox, interact, IntSlider
import numpy as np

In [2]:
def computeDihedralAngle(mol, indices, conf_id = 0):
    c = mol.GetConformer(conf_id)
    pos = []
    for idx in indices:
        pos.append(c.GetAtomPosition(idx))
    phi = rdGeometry.ComputeSignedDihedralAngle(pos[0], pos[1], pos[2],pos[3])*180/np.pi
    return phi

def dihedralsFromConformers(mol, dih_list):
    mtx = np.zeros((len(list(mol.GetConformers())), len(dih_list)))
    for i, c in enumerate(list(mol.GetConformers())):
        for k, indices in enumerate(dih_list):
            v = computeDihedralAngle(mol, indices, conf_id = i)
            mtx[i,k] = v
    return mtx

In [3]:
def load_conformers(mol):
    """
    """
    view = nv.NGLWidget()
    #view.add_component(str(Path(base_path, "protein.pdb")))
    nconfs = len(mol.GetConformers())
    comps = []
    for i in range(nconfs):
        s = nv.RdkitStructure(mol, conf_id = i)
        #view.add_structure(s)
        c = view.add_component(s)
        comps.append(c)
    print(comps)
#    for p in probes:
#        view.add_component(str(Path(base_path, f"{p}.ccp4")))
    return view, comps

class LigandViewer():
    
    def __init__(self, mol):
        self.mol = mol
        #self.view, self.comps = load_conformers(mol)
        self.view = nv.NGLWidget()
        s = nv.RdkitStructure(self.mol, conf_id = 1)
        self.c = self.view.add_component(s)
        
    def show_conformer(self, idx):
        """
        Given the name of the loaded file (or a partial string),
        returns the component index if loaded
        """
#        print(idx)
        if self.c is not None:
            self.view.remove_component(self.c)
        new_id = idx.new
        s = nv.RdkitStructure(self.mol, conf_id = new_id)
        self.c = self.view.add_component(s)
        
        #all_components = self.view._ngl_component_names
        #print(all_components)
#        print(self.comps[idx])
#        return self.comps[idx]
#        component_idx = all_components.index([i for i in all_components 
#                                              if component_name in i][0])
#        return component_idx

    def threshold_isosurface(self, idx):
        """
        Controls the representation of the map for specific probe
        :param probe_name: string, one of 'donor', 'acceptor', 'apolar'
        :param threshold: threshold value for displaying the hotspot map
        """
        colour_dict = {'donor': 'blue',
                       'acceptor': 'red',
                       'apolar': '#FFF176'}
        comp = self.get_component_index(idx)
        repr_params = [{'type': 'surface',
                        'params': {'opacity': 0.4,
                                   'isolevelType': 'value',
#                                   'isolevel': threshold,
                                   'color': '#FFF176'}}]
        self.view.set_representations(repr_params, component=comp)

In [4]:
ligand = Chem.MolFromMolFile('../data/ligand_md/ligand.sdf', removeHs = False)
_ = AllChem.EmbedMultipleConfs(ligand, numConfs = 20)

In [5]:
lst = []
lst.append( [7,8,9,10] )
lst.append( [8,9,10,11] )
lst.append( [9,10,11,14] )
lst.append( [14,11,12,17] )
mtx = dihedralsFromConformers(ligand, lst)

In [6]:
viewer = LigandViewer(ligand)

In [7]:
slider = IntSlider(min=1,max=len(list(ligand.GetConformers()))-1)

slider.observe(viewer.show_conformer, 'value')
VBox([viewer.view, slider])

In [ ]:
nv.__version__

In [ ]:
import ipywidgets
ipywidgets.__version__

In [8]:
nv.demo()

NGLWidget()

In [ ]:
rdGeometry.ComputeDihedralAngle